In [3]:
import os
import numpy as np
np.random.seed(43)

In [4]:
setup_string = '''
# General setup
open morph-open-closed/open-closed.pdb
center
turn y 60

# Open bound structure get position of the ligand, missing in the morph
open 1ake
delete #1:.B
addh spec #1&protein hbond false
mmaker #0.1 #1

# Window appearance
windowsize 800 600
lighting reflectivity 0
set bg_color white
set silhouette
set silhouette_width 1.5

# Model appearance
color grey

# Add caption
2dlabel create title text "Binding and release of substrate leads to directional motion" xpos 0.1 ypos 0.92 color black

# Play movie sequence
2dlabel change title visibility show

# Hide everything
~ribbon; ~display
# Except ligand
display #1:AP5; color red #1:AP5; color byhet #1:AP5
'''

In [5]:
def pause_and_fade(pause_length, total_frames, file):
    file.write('# Fade out...\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format((pause + 1)* transparency_increment)
                  )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    file.write('~display #1:AP5\n')
    return total_frames

In [6]:
def pause_and_fade_in(pause_length, total_frames, file):
    file.write('# Fade in...\n')
    file.write('display #1:AP5\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format(100 - ((pause + 1)* transparency_increment))
        )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )

    return total_frames


def pause(pause_length, total_frames, file):
    for pause in np.arange(pause_length):
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    return total_frames

In [7]:
def add_bump(positive):
    sigma = 2
    if positive:
        mu = 6.0
    else:
        mu = -6.0
    rotation = sigma * np.random.randn(1) + mu
    return rotation[0]

In [26]:
def rotate_forward(models, residues, positives, 
                   list_of_previous_positions, file, total_frames):
    #
    positions = np.empty(( len(residues), len(models) + 1))
    positions[:, 0] = list_of_previous_positions
    print('Setting previous positions...')
    print('{}'.format(list_of_previous_positions[0]))
    #
    for (model_index, model) in enumerate(models, 1):
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues): 
            # Now keep incrementing this!
            positions[residue_index, model_index] = \
                            positions[residue_index, model_index - 1] + \
                            add_bump(positives[residue_index])

            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            colors = ['blue', 'green', 'purple', 'orange', 'yellow']
            file.write('rotation 2 {} 1; wait 1\n'.format(positions[residue_index, model_index]))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')

            #
            print('Frame {} → {}'.format(model + 1, positions[residue_index, model_index]))
            additional_rotation = positions[residue_index, model_index] - \
                                  list_of_previous_positions[residue_index, -model_index]
            print('Frame {} → {}, which is a rotation of additional {}'.format(model + 1, 
                                                                               positions[residue_index, model_index],
                                                                               additional_rotation))            

            #
            
        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
        )

    return positions, total_frames

In [27]:
def rotate_backward(models, residues, positives, file, forward_rotations, total_frames):
    #
    positions = np.empty(( len(residues), len(models) + 1))
    positions[:, 0] = forward_rotations[:, -1]
    #
    # Now keep incrementing this!
    #
    for (model_index, model) in enumerate(models[::-1], 1):
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues):
            if model_index == 1:
                print("This angle's last position is... {}".format(positions[residue_index, 0]))
            
            positions[residue_index, model_index] = \
                            positions[residue_index, model_index - 1] + \
                            add_bump(positives[residue_index])

            #
            additional_rotation = positions[residue_index, model_index] - \
                                  forward_rotations[residue_index, -model_index]
            print('Frame {} → {}, which is a rotation of additional {}'.format(model + 1, 
                                                                               positions[residue_index, model_index],
                                                                               additional_rotation))            
            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            colors = ['blue', 'green', 'purple', 'orange', 'yellow']
            file.write('rotation 2 {} 1; wait 1\n'.format(additional_rotation))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')


        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
            )
    
    return positions, total_frames

In [28]:
# residues = [20, 50, 120, 141, 200]
residues = [20]
positives = [True, False, False, True, True]
pause_length = 20
morphs = np.arange(30)
total_frames = 0


with open('morph_setup.cmd', 'w') as file:
    file.write(setup_string)

    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives,
        list_of_previous_positions=[0] * len(residues),
        file=file, 
        total_frames=total_frames)
    
    print(forward_rotations)
    
    total_frames = pause_and_fade(5, total_frames, file)
    
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        file=file, 
        forward_rotations=forward_rotations, 
        total_frames=total_frames)
    
    total_frames = pause_and_fade_in(5, total_frames, file)
    
    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives, 
        list_of_previous_positions=backward_rotations[:, -1],
        file=file, 
        total_frames=total_frames)

#     backward_rotations, total_frames = rotate_backward(
#         morphs, 
#         residues, 
#         positives, 
#         file, 
#         backward_rotations, 
#         total_frames)

#     total_frames = pause(5, total_frames, file)

Setting previous positions...
0
Frame 1 → 7.732637341695758


TypeError: list indices must be integers or slices, not tuple

```
ffmpeg -r 10 -i open-closed-%03d.png -y -vf format=yuv420p morph.mp4
```

I probably need to create a function that goes forward, just starting with an initial value -- like I did for the backwards process, but without stepping through the frames backwards. In fact, I should just make this the general going forwards function and pass an array of zeros for the first trip forwards.